## FULL CODE

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

# Setup WebDriver
print("🚀 Launching browser...")
service = Service('C:/chromedriver-win64/chromedriver.exe')  # Update path if needed
driver = webdriver.Chrome(service=service)

try:
    print("🌐 Opening CBSL exchange rates page...")
    driver.get('https://www.cbsl.gov.lk/en/rates-and-indicators/exchange-rates/daily-indicative-exchange-rates')
    wait = WebDriverWait(driver, 50)

    # Step 1: Locate and switch through iframes to find the one with the date input
    print("🧭 Locating and checking all iframes...")
    iframe_list = driver.find_elements(By.TAG_NAME, "iframe")
    print(f"🔍 Found {len(iframe_list)} iframe(s)")

    date_iframe = None
    for i, iframe in enumerate(iframe_list):
        print(f"🔁 Switching to iframe {i+1}")
        driver.switch_to.default_content()
        driver.switch_to.frame(iframe)
        try:
            date_input = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "txtStart"))
            )
            date_iframe = iframe
            print(f"✅ Date input field found in iframe {i+1}")
            break
        except:
            print(f"❌ Date input field not found in iframe {i+1}")
            continue

    if date_iframe is None:
        raise Exception("❌ Date input field not found in any iframe!")

    # Step 2: Interact with the date picker
    print("📅 Interacting with the date picker...")
    date_input.click()
    time.sleep(1)

    caption = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//td[@class='dp_caption']"))
    )
    caption.click()
    print("✅ Caption clicked")

    previous_month_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//td[@class='dp_previous']"))
    )
    previous_month_button.click()
    print("✅ Previous month clicked")

    january_selector = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//td[@class='dp_month_0']"))
    )
    january_selector.click()
    print("✅ January selected")

    day_selector = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//td[text()='1']"))
    )
    day_selector.click()
    print("✅ Day 1 selected")

    print("📅 Date selected successfully!")

    # Step 3: Uncheck EUR and JPY (same iframe)
    print("🔍 Locating EUR checkbox...")
    eur_checkbox = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@value='EUR~Euro']")))
    if eur_checkbox.is_selected():
        eur_checkbox.click()
        print("☑️ Unchecked EUR")
    else:
        print("✅ EUR already unchecked")

    print("🔍 Locating JPY checkbox...")
    jpy_checkbox = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@value='JPY~Japanese Yen']")))
    if jpy_checkbox.is_selected():
        jpy_checkbox.click()
        print("☑️ Unchecked JPY")
    else:
        print("✅ JPY already unchecked")

    # Step 4: Submit the form
    print("📤 Submitting the form...")
    submit_button = wait.until(EC.element_to_be_clickable((By.NAME, 'submit_button')))
    submit_button.click()

    # Step 5: Wait for results and extract
    print("⏳ Waiting for results...")
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'table')))
    time.sleep(2)
    print("✅ Results table loaded")

    table = driver.find_element(By.XPATH, "//table[contains(@class,'table')]")
    rows = table.find_elements(By.TAG_NAME, "tr")

    if not rows:
        raise Exception("❌ No rows found.")

    headers = [th.text.strip() for th in rows[0].find_elements(By.TAG_NAME, "th")]
    data = [[td.text.strip() for td in row.find_elements(By.TAG_NAME, "td")] for row in rows[1:]]

    df = pd.DataFrame(data, columns=headers)
    print("📄 DataFrame preview:")
    print(df.head())

    df.to_csv("cbsl_exchange_rates.csv", index=False)
    print("💾 Data saved to cbsl_exchange_rates.csv")

finally:
    print("🧹 Closing browser...")
    driver.quit()


🚀 Launching browser...
🌐 Opening CBSL exchange rates page...
🧭 Locating and checking all iframes...
🔍 Found 3 iframe(s)
🔁 Switching to iframe 1
❌ Date input field not found in iframe 1
🔁 Switching to iframe 2
❌ Date input field not found in iframe 2
🔁 Switching to iframe 3
✅ Date input field found in iframe 3
📅 Interacting with the date picker...
✅ Caption clicked
✅ Previous month clicked
✅ January selected
✅ Day 1 selected
📅 Date selected successfully!
🔍 Locating EUR checkbox...
☑️ Unchecked EUR
🔍 Locating JPY checkbox...
☑️ Unchecked JPY
📤 Submitting the form...
⏳ Waiting for results...
✅ Results table loaded
📄 DataFrame preview:
         Date 1 USD -> LKR 1 LKR -> USD
0  2025-04-21     299.0786       0.0033
1  2025-04-17     298.6186       0.0033
2  2025-04-16     298.1380       0.0034
3  2025-04-11     298.8721       0.0033
4  2025-04-10     301.2049       0.0033
💾 Data saved to cbsl_exchange_rates.csv
🧹 Closing browser...
